# Objective:

This is a first attempt at creating a simple GAN model following the documentation of *Deep Learning with Pytorch* by Vishnu Subramanian and this Medium article: https://medium.com/@devnag/generative-adversarial-networks-gans-in-50-lines-of-code-pytorch-e81b79659e3f

Certain model parameters and structure will be based off of the pix2pix model implemented in *Image-to-Image Translation with Conditional Adversarial Networks* by Isola, et al.  


In [1]:
import pandas as pd 
import numpy as np

from pathlib import Path
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import itertools
import time 
import cv2
import os

from PIL import Image
import math

## 1. Load re-sized images and their corresponding sketches 

In [2]:
!pwd

/home/ec2-user/couch_gan


In [3]:
## code to resize images 

# def resize_(path, sz=250):
#     """
#     resizes a image based on a center crop and to scale horizontal and vertical axis to a size 
#     """
#     im = cv2.imread(str(path))
#     return cv2.resize(im, (sz, sz))   

# def resize_all_images(orig_path, new_path):
#     """
#     resizes an entire directory and saves to a new directory 
#     """
#     for f in (orig_path).iterdir():  # iterating through a directory 
#         img = resize_(f)  # crop each file 
#         path = new_path/f.name # saves file path 
#         cv2.imwrite(str(path), img)  # writes out image to the file path 
        
# os.makedirs("./data/resized_250/original", exist_ok=True)
# os.makedirs("./data/resized_250/sketch", exist_ok=True)

# resize_all_images(orig_path=Path('./data/og_resized'), new_path=Path('./data/resized_250/original'))
# resize_all_images(orig_path=Path('./data/sketch_resized/'), new_path=Path('./data/resized_250/sketch'))

In [4]:
# assign paths 
# real_img_path = Path('./data/og_resized')
# sketch_img_path = Path('./data/sketch_resized')
real_img_path = Path('./data/resized_250/original/')
sketch_img_path = Path('./data/resized_250/sketch/')
fake_img_path = Path('./data/fake')

In [5]:
# get files into list  
real_img_files = [f for f in real_img_path.iterdir()]
sketch_img_files = [f for f in sketch_img_path.iterdir()]

## 2. Get image names of training and test files (assigned by Kaggle)

In [6]:
test_file_names = [f'00000{i}.jpg' for i in range(203, 303)]

In [7]:
# assign paths to appropriate lists 
train_real_paths = []
train_sketch_paths = []
test_real_paths = []
test_sketch_paths = []

for f in real_img_files:
    if f.parts[-1] in test_file_names:
        test_real_paths.append(f)
    else:
        train_real_paths.append(f)

for f in sketch_img_files:
    if f.parts[-1] in test_file_names:
        test_sketch_paths.append(f)
    else:
        train_sketch_paths.append(f)        

In [8]:
# remove any inapporpiate files like .DS_Store 
train_sketch_paths = [file for file in train_sketch_paths if not file.parts[-1].startswith('.')]
test_real_paths = [file for file in test_real_paths if not file.parts[-1].startswith('.')]
test_sketch_paths = [file for file in test_sketch_paths if not file.parts[-1].startswith('.')]
train_real_paths = [file for file in train_real_paths if not file.parts[-1].startswith('.')]


In [9]:
# sanity check that the order of real and sketch paths are the same 
counter = 0
for r, f in zip(train_real_paths, train_sketch_paths):
    if r.parts[-1] == r.parts[-1]:
        counter += 1
assert(counter == 900)

counter = 0
for r, f in zip(test_real_paths, test_sketch_paths):
    if r.parts[-1] == r.parts[-1]:
        counter += 1
assert(counter == 100)

## 3. Couch Dataset  and Dataloaders 

Note: No data augmentation 

In [10]:
def getX(path):
    x = cv2.imread(str(path)).astype(np.float32)  # reading image 
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)/255  # convert from BGR to RGB
    return x 

In [11]:
def normalize(im):
    """Normalizes images """
    imagenet_stats = np.array([[0.5, 0.5, 0.5], [0.5, 0.5, 0.5]])
    return (im - imagenet_stats[0])/imagenet_stats[1]

In [12]:
class CouchDataset(Dataset):
    def __init__(self, real_files, sketch_files):
        self.real_files = real_files
        self.sketch_files = sketch_files
        
    def __len__(self):
        return len(self.real_files)  # should be same number as sketch files 
        
    def __getitem__(self, idx):
        real_path = self.real_files[idx]
        sketch_path = self.sketch_files[idx]
        img_name = sketch_path.parts[-1]
        
        x_real = getX(real_path)
        x_sketch = getX(sketch_path)
        
        # normalize 
        x_real = normalize(x_real)
        x_sketch = normalize(x_sketch)
        
        # roll axis (channels, height, width)
        x_real = np.rollaxis(x_real, 2)
        x_sketch = np.rollaxis(x_sketch, 2)
        
        return x_sketch, x_real
        

In [13]:
# make datasets 
train_couch_ds = CouchDataset(real_files=train_real_paths, sketch_files=train_sketch_paths)
test_couch_ds = CouchDataset(real_files=test_real_paths, sketch_files=test_sketch_paths)



In [14]:
# inspect content 
x_s, x_r = train_couch_ds[20]


In [15]:
x_s, x_r

(array([[[ 1.        ,  1.        ,  1.        , ...,  0.97647059,
           1.        ,  0.97647059],
         [ 1.        ,  1.        ,  1.        , ..., -0.84313725,
          -0.89019608,  0.42745101],
         [ 1.        ,  1.        ,  1.        , ...,  0.78039217,
           0.78039217,  0.94509804],
         ...,
         [ 1.        ,  1.        ,  1.        , ...,  1.        ,
           1.        ,  1.        ],
         [ 1.        ,  1.        ,  1.        , ...,  1.        ,
           1.        ,  1.        ],
         [ 1.        ,  1.        ,  1.        , ...,  1.        ,
           1.        ,  1.        ]],
 
        [[ 1.        ,  1.        ,  1.        , ...,  0.97647059,
           1.        ,  0.97647059],
         [ 1.        ,  1.        ,  1.        , ..., -0.84313725,
          -0.89019608,  0.42745101],
         [ 1.        ,  1.        ,  1.        , ...,  0.78039217,
           0.78039217,  0.94509804],
         ...,
         [ 1.        ,  1.       

In [16]:

x_s.shape, x_r.shape

((3, 250, 250), (3, 250, 250))

## 4. Model  

In [17]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch.nn as nn
import torch.nn.functional as F
import torch

from torchvision.models import resnet18

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

In [18]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [154]:
# # based on Pix2pix GAN 
# ## https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/pix2pix/models.py

class UNetDown(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, kernel_size=3, stride=2, padding=1, bias=False)]
#         layers = [nn.Conv2d(in_size, out_size, kernel_size=3, padding=1, bias=False)] # using https://github.com/xiaopeng-liao/Pytorch-UNet/blob/master/unet/unet_parts.py

        if normalize:
            layers.append(nn.InstanceNorm2d(out_size))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class UNetUp(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_size, out_size, kernel_size=3, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(out_size),
            nn.ReLU(inplace=True),
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))

        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        print("original x shape", x.shape)
         
        diffX = skip_input.size()[2] - x.size()[2]
        diffY = skip_input.size()[3] - x.size()[3]
        x1 = F.pad(x, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        
        print("x1 shape", x.shape)
        
        x = torch.cat((x1, skip_input), 1)
        print("concatenated x shape", x.shape)

        return x


class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(GeneratorUNet, self).__init__()

        self.down1 = UNetDown(in_channels, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512, dropout=0.5)
        self.down5 = UNetDown(512, 512, dropout=0.5)
        self.down6 = UNetDown(512, 512, dropout=0.5)
        self.down7 = UNetDown(512, 512, dropout=0.5)
        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)

        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 512, dropout=0.5)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128)
        self.up7 = UNetUp(256, 64)

        self.final = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, out_channels, 3, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        # U-Net generator with skip connections from encoder to decoder
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)
        print("x", x.shape)
        print("d1",d1.shape)
        print("d2",d2.shape)
        print("d3",d3.shape)
        print("d4",d4.shape)
        print("d5",d5.shape)
        print("d6",d6.shape)
        print("d7",d7.shape)
        print("d8",d8.shape)
        
        u1 = self.up1(d8, d7)
        print("u1", u1.shape)
        u2 = self.up2(u1, d6)
        print("u2", u2.shape)
        u3 = self.up3(u2, d5)
        print("u3", u3.shape)

        u4 = self.up4(u3, d4)
        print("u4", u4.shape)

        u5 = self.up5(u4, d3)
        print("u5", u5.shape)

        u6 = self.up6(u5, d2)
        print("u6", u6.shape)

        u7 = self.up7(u6, d1)
        print("u7", u7.shape)

        return self.final(u7)




In [155]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 3, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels * 2, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 3, padding=1, bias=False)
        )

    def forward(self, img_A, img_B):
        # Concatenate image and condition image by channels to produce input
        img_input = torch.cat((img_A, img_B), 1)
        return self.model(img_input)

## 5. Training 

In [156]:
# Create sample and checkpoint directories
os.makedirs("checkpoint", exist_ok=True)
os.makedirs("saved_models", exist_ok=True)
os.makedirs("fake_imgs", exist_ok=True)


#### Debugging 

In [157]:
# create small data loaders 
# batch_size = 10
batch_size = 1
train_dl = DataLoader(train_couch_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_couch_ds, batch_size=batch_size)

In [158]:
train_x_s, train_x_r = next(iter(train_dl))

In [159]:
train_x_s.shape

torch.Size([1, 3, 250, 250])

In [160]:
train_x_r.shape

torch.Size([1, 3, 250, 250])

In [161]:
input_shape = (3, 250, 250)  # TODO: dynamically grab this 

# used default parameters for this 
latent_dim = 8 
lr = 0.01
b1 = 0.5  # adam: decay of first order momentum of gradient
b2 = 0.999  # adam: decay of first order momentum of gradient


In [162]:
# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel = 100

In [163]:
# Calculate output of image discriminator (PatchGAN)
img_height = 250
img_width = 250
patch = (1, img_height // 2 ** 4, img_width // 2 ** 4)


In [164]:
# Initialize generator and discriminator and loss functions 
generator = GeneratorUNet().cuda()
discriminator = Discriminator().cuda()
criterion_GAN = torch.nn.MSELoss().cuda()
criterion_pixelwise = torch.nn.L1Loss().cuda()

In [165]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (12): Conv2d(512, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
)

In [166]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))


In [167]:
# Tensor type
Tensor = torch.cuda.FloatTensor 

In [168]:
def sample_images(batches_done):
    """Saves a generated sample from the validation set"""
    x_sketch, x_real = next(iter(test_dl))
    x_sketch_var = Variable(x_sketch.type(Tensor))
    x_real_var = Variable(x_real.type(Tensor))
    fake_from_sketch = generator(x_sketch_var)
    
    img_sample = torch.cat((x_sketch_var.data, fake_from_sketch.data, x_real_var.data), -2)
    save_image(img_sample, "fake_imgs/%s/%s.png" % ("pix2pix", batches_done), nrow=5, normalize=True)
    
    
    

In [169]:
for i, batch in enumerate(train_dl):
    # model inputs 
    x_sketch_var = Variable(batch[0].type(Tensor))
    x_real_var = Variable(batch[1].type(Tensor))
    
    # adveserial ground truths 
    # iffy about size; should it be the number of channels
    valid = Variable(Tensor(np.ones((x_sketch_var.size(0), *patch))), requires_grad=False)
    fake = Variable(Tensor(np.zeros((x_sketch_var.size(0), *patch))), requires_grad=False)
    
    # train generators 
    optimizer_G.zero_grad()
    
    # GAN loss
    fake_sketch = generator(x_real_var)
    

x torch.Size([1, 3, 250, 250])
d1 torch.Size([1, 64, 125, 125])
d2 torch.Size([1, 128, 63, 63])
d3 torch.Size([1, 256, 32, 32])
d4 torch.Size([1, 512, 16, 16])
d5 torch.Size([1, 512, 8, 8])
d6 torch.Size([1, 512, 4, 4])
d7 torch.Size([1, 512, 2, 2])
d8 torch.Size([1, 512, 1, 1])
original x shape torch.Size([1, 512, 1, 1])
x1 shape torch.Size([1, 512, 1, 1])
concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1, 512, 15, 15])
concatenated x shape torch.Size([1, 1024, 16, 16])
u4 torch.Size([1, 1024, 16, 16])
original x shape torch.Size([1, 256, 31, 31])
x1 shape torch.Size([1, 25

x1 shape torch.Size([1, 128, 63, 63])
concatenated x shape torch.Size([1, 256, 63, 63])
u6 torch.Size([1, 256, 63, 63])
original x shape torch.Size([1, 64, 125, 125])
x1 shape torch.Size([1, 64, 125, 125])
concatenated x shape torch.Size([1, 128, 125, 125])
u7 torch.Size([1, 128, 125, 125])
x torch.Size([1, 3, 250, 250])
d1 torch.Size([1, 64, 125, 125])
d2 torch.Size([1, 128, 63, 63])
d3 torch.Size([1, 256, 32, 32])
d4 torch.Size([1, 512, 16, 16])
d5 torch.Size([1, 512, 8, 8])
d6 torch.Size([1, 512, 4, 4])
d7 torch.Size([1, 512, 2, 2])
d8 torch.Size([1, 512, 1, 1])
original x shape torch.Size([1, 512, 1, 1])
x1 shape torch.Size([1, 512, 1, 1])
concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.S

x torch.Size([1, 3, 250, 250])
d1 torch.Size([1, 64, 125, 125])
d2 torch.Size([1, 128, 63, 63])
d3 torch.Size([1, 256, 32, 32])
d4 torch.Size([1, 512, 16, 16])
d5 torch.Size([1, 512, 8, 8])
d6 torch.Size([1, 512, 4, 4])
d7 torch.Size([1, 512, 2, 2])
d8 torch.Size([1, 512, 1, 1])
original x shape torch.Size([1, 512, 1, 1])
x1 shape torch.Size([1, 512, 1, 1])
concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1, 512, 15, 15])
concatenated x shape torch.Size([1, 1024, 16, 16])
u4 torch.Size([1, 1024, 16, 16])
original x shape torch.Size([1, 256, 31, 31])
x1 shape torch.Size([1, 25

x torch.Size([1, 3, 250, 250])
d1 torch.Size([1, 64, 125, 125])
d2 torch.Size([1, 128, 63, 63])
d3 torch.Size([1, 256, 32, 32])
d4 torch.Size([1, 512, 16, 16])
d5 torch.Size([1, 512, 8, 8])
d6 torch.Size([1, 512, 4, 4])
d7 torch.Size([1, 512, 2, 2])
d8 torch.Size([1, 512, 1, 1])
original x shape torch.Size([1, 512, 1, 1])
x1 shape torch.Size([1, 512, 1, 1])
concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1, 512, 15, 15])
concatenated x shape torch.Size([1, 1024, 16, 16])
u4 torch.Size([1, 1024, 16, 16])
original x shape torch.Size([1, 256, 31, 31])
x1 shape torch.Size([1, 25

x torch.Size([1, 3, 250, 250])
d1 torch.Size([1, 64, 125, 125])
d2 torch.Size([1, 128, 63, 63])
d3 torch.Size([1, 256, 32, 32])
d4 torch.Size([1, 512, 16, 16])
d5 torch.Size([1, 512, 8, 8])
d6 torch.Size([1, 512, 4, 4])
d7 torch.Size([1, 512, 2, 2])
d8 torch.Size([1, 512, 1, 1])
original x shape torch.Size([1, 512, 1, 1])
x1 shape torch.Size([1, 512, 1, 1])
concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1, 512, 15, 15])
concatenated x shape torch.Size([1, 1024, 16, 16])
u4 torch.Size([1, 1024, 16, 16])
original x shape torch.Size([1, 256, 31, 31])
x1 shape torch.Size([1, 25

...

concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1, 512, 15, 15])
concatenated x shape torch.Size([1, 1024, 16, 16])
u4 torch.Size([1, 1024, 16, 16])
original x shape torch.Size([1, 256, 31, 31])
x1 shape torch.Size([1, 256, 31, 31])
concatenated x shape torch.Size([1, 512, 32, 32])
u5 torch.Size([1, 512, 32, 32])
original x shape torch.Size([1, 128, 63, 63])
x1 shape torch.Size([1, 128, 63, 63])
concatenated x shape torch.Size([1, 256, 63, 63])
u6 torch.Size([1, 256, 63, 63])
original x shape torch.Size([1, 64, 125, 125])
x1 shape torch.Size([1, 64, 125, 125])
concatenated x

concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1, 512, 15, 15])
concatenated x shape torch.Size([1, 1024, 16, 16])
u4 torch.Size([1, 1024, 16, 16])
original x shape torch.Size([1, 256, 31, 31])
x1 shape torch.Size([1, 256, 31, 31])
concatenated x shape torch.Size([1, 512, 32, 32])
u5 torch.Size([1, 512, 32, 32])
original x shape torch.Size([1, 128, 63, 63])
x1 shape torch.Size([1, 128, 63, 63])
concatenated x shape torch.Size([1, 256, 63, 63])
u6 torch.Size([1, 256, 63, 63])
original x shape torch.Size([1, 64, 125, 125])
x1 shape torch.Size([1, 64, 125, 125])
concatenated x

original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1, 512, 15, 15])
concatenated x shape torch.Size([1, 1024, 16, 16])
u4 torch.Size([1, 1024, 16, 16])
original x shape torch.Size([1, 256, 31, 31])
x1 shape torch.Size([1, 256, 31, 31])
concatenated x shape torch.Size([1, 512, 32, 32])
u5 torch.Size([1, 512, 32, 32])
original x shape torch.Size([1, 128, 63, 63])
x1 shape torch.Size([1, 128, 63, 63])
concatenated x shape torch.Size([1, 256, 63, 63])
u6 torch.Size([1, 256, 63, 63])
original x shape torch.Size([1, 64, 125, 125])
x1 shape torch.Size([1, 64, 125, 125])
concatenated x shape torch.Size([1, 128, 125, 125])
u7 torch.Size([1, 128, 125, 125])
x torch.

x torch.Size([1, 3, 250, 250])
d1 torch.Size([1, 64, 125, 125])
d2 torch.Size([1, 128, 63, 63])
d3 torch.Size([1, 256, 32, 32])
d4 torch.Size([1, 512, 16, 16])
d5 torch.Size([1, 512, 8, 8])
d6 torch.Size([1, 512, 4, 4])
d7 torch.Size([1, 512, 2, 2])
d8 torch.Size([1, 512, 1, 1])
original x shape torch.Size([1, 512, 1, 1])
x1 shape torch.Size([1, 512, 1, 1])
concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1, 512, 15, 15])
concatenated x shape torch.Size([1, 1024, 16, 16])
u4 torch.Size([1, 1024, 16, 16])
original x shape torch.Size([1, 256, 31, 31])
x1 shape torch.Size([1, 25

original x shape torch.Size([1, 64, 125, 125])
x1 shape torch.Size([1, 64, 125, 125])
concatenated x shape torch.Size([1, 128, 125, 125])
u7 torch.Size([1, 128, 125, 125])
x torch.Size([1, 3, 250, 250])
d1 torch.Size([1, 64, 125, 125])
d2 torch.Size([1, 128, 63, 63])
d3 torch.Size([1, 256, 32, 32])
d4 torch.Size([1, 512, 16, 16])
d5 torch.Size([1, 512, 8, 8])
d6 torch.Size([1, 512, 4, 4])
d7 torch.Size([1, 512, 2, 2])
d8 torch.Size([1, 512, 1, 1])
original x shape torch.Size([1, 512, 1, 1])
x1 shape torch.Size([1, 512, 1, 1])
concatenated x shape torch.Size([1, 1024, 2, 2])
u1 torch.Size([1, 1024, 2, 2])
original x shape torch.Size([1, 512, 3, 3])
x1 shape torch.Size([1, 512, 3, 3])
concatenated x shape torch.Size([1, 1024, 4, 4])
u2 torch.Size([1, 1024, 4, 4])
original x shape torch.Size([1, 512, 7, 7])
x1 shape torch.Size([1, 512, 7, 7])
concatenated x shape torch.Size([1, 1024, 8, 8])
u3 torch.Size([1, 1024, 8, 8])
original x shape torch.Size([1, 512, 15, 15])
x1 shape torch.Size([1

In [170]:
fake_sketch.shape

torch.Size([1, 3, 250, 250])

In [171]:
x_real_var[-1].shape

torch.Size([3, 250, 250])

In [172]:
fake_sketch

tensor([[[[-0.0855, -0.2416, -0.2673,  ..., -0.0914, -0.1170, -0.1144],
          [-0.2794, -0.3694, -0.0270,  ..., -0.2342, -0.1471, -0.2559],
          [-0.1670, -0.1224,  0.0438,  ..., -0.0761,  0.0379, -0.2150],
          ...,
          [-0.1634, -0.0601, -0.1789,  ..., -0.0598, -0.0920, -0.3935],
          [-0.2696, -0.1819, -0.0718,  ..., -0.4617, -0.4633, -0.2311],
          [-0.0484, -0.2504,  0.0756,  ...,  0.0237, -0.0887, -0.2251]],

         [[-0.3044, -0.5313, -0.3909,  ..., -0.0413, -0.2055, -0.3176],
          [-0.3244, -0.2822, -0.2059,  ..., -0.5100, -0.3339, -0.1033],
          [-0.1541, -0.3788, -0.2281,  ..., -0.5380, -0.2482,  0.0569],
          ...,
          [-0.1766, -0.4773, -0.5340,  ..., -0.5267, -0.6826, -0.0823],
          [-0.1931, -0.3403, -0.3391,  ..., -0.3629, -0.3262,  0.0774],
          [-0.3041, -0.3271, -0.3072,  ..., -0.1692, -0.0656,  0.1215]],

         [[-0.0710, -0.2072, -0.2852,  ..., -0.4194, -0.2950, -0.0503],
          [ 0.4382, -0.1316, -

In [173]:
x_real_var[-1]

tensor([[[ 0.8510,  0.8510,  0.8510,  ...,  0.9843,  0.9922,  1.0000],
         [ 0.8510,  0.8510,  0.8510,  ...,  0.9843,  0.9922,  1.0000],
         [ 0.8431,  0.8431,  0.8431,  ...,  0.9843,  0.9922,  1.0000],
         ...,
         [-0.3098, -0.3412, -0.3647,  ...,  0.9686,  0.9922,  1.0000],
         [-0.3804, -0.3882, -0.4275,  ...,  0.9686,  0.9922,  1.0000],
         [-0.4196, -0.4275, -0.4667,  ...,  0.9686,  0.9922,  1.0000]],

        [[ 0.8510,  0.8510,  0.8510,  ...,  0.9843,  0.9922,  1.0000],
         [ 0.8510,  0.8510,  0.8510,  ...,  0.9843,  0.9922,  1.0000],
         [ 0.8431,  0.8431,  0.8431,  ...,  0.9843,  0.9922,  1.0000],
         ...,
         [-0.3569, -0.3882, -0.3961,  ...,  0.9686,  0.9922,  1.0000],
         [-0.4510, -0.4353, -0.4588,  ...,  0.9686,  0.9922,  1.0000],
         [-0.4902, -0.4824, -0.4980,  ...,  0.9686,  0.9922,  1.0000]],

        [[ 0.8667,  0.8667,  0.8667,  ...,  0.9843,  0.9922,  1.0000],
         [ 0.8667,  0.8667,  0.8667,  ...,  0

## Comment for next time:
work on the rest of the debugging portion involving the discriminator  